In [1]:
import numpy as np
import pandas as pd

from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time
from tqdm import tqdm

In [2]:
# 유튜브 - 사나고 영상 페이지
url = "https://www.youtube.com/watch?v=vEIjQYiiOvM&t=8s"

In [3]:
# 테스트 브라우저 열기
driver = wb.Chrome()
driver.get(url)

In [4]:
driver.maximize_window()

In [5]:
# 모든 댓글을 수집하기에는 시간이 많이 걸림...
# 5번만 스크롤 하강 수행

for i in range(5) : 
    body = driver.find_element(By.TAG_NAME, "body")
    body.send_keys(Keys.END)
    time.sleep(2)

In [6]:
# 지정할 때 copy selector를 쓰던, 수기로 작성하던 항상 결과 확인!
driver.find_elements(By.CSS_SELECTOR, "yt-attributed-string#content-text")[0].text

'엄마 엄마! 나 사나고 사줘~!'

In [9]:
#content-text

In [7]:
# 리뷰와 좋아요 요소 지정
reviews=driver.find_elements(By.CSS_SELECTOR, "yt-attributed-string#content-text")
likes = driver.find_elements(By.CSS_SELECTOR, "span#vote-count-middle")

In [8]:
len(reviews), len(likes)

(100, 100)

In [9]:
int(float(likes[1].text.replace("천",""))*1000)

3000

In [10]:
# 1. 좋아요가 없으면 0으로 채우기
# 2. 단위로 글자가 들어가 있는 경우 변환

reviews_list=[]
like_list=[]

#추출
for i in range(len(reviews)):
    reviews_list.append(reviews[i].text) #해당 리뷰 쌓아주기
    # 좋아요가 없으면 0으로 채우기
    if likes[i].text == "": #해당 리뷰에 좋아요가 없을때
        like_list.append(0) #0으로 대체해서 쌓아주기
    else:
        try: #좋아요를 형변환(정수)해서 쌓아주기
            like_list.append(int(likes[i].text)) #정수로 형변환 시도해보기!
        except: #좋아요가 숫자값이 아닌 경우!
            like_list.append(int(float(likes[i].text.replace("천",""))*1000))

In [11]:
re_dict={
    "리뷰":reviews_list,
    "좋아요 수":like_list
}
pd.DataFrame(re_dict)

,리뷰,좋아요 수
0,엄마 엄마! 나 사나고 사줘~!,635
1,곧 문의 메일이 폭주할 사나고님입니다,3000
2,11:24 치마 훌렁~ 훌렁~ 하는고 왤케 귀여우셬ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 목소리 중독적,160
3,왕크니까 왕귀엽습니다,410
4,11:25 훌렁 너무 웃기네 ㅋㅋㅋㅋㅋㅋㅋ,1500
...,...,...
95,티니핑 마니아로써 더 높은 퀄리티와 반짝이를 요청합니다...!! 다른 것도 해주세...,2
96,그냥 하츄핑이 아닌 플로라 하츄핑입니다..,4
97,확대해주셔서 감사해요 대리만족!!!!!\n제가 좋아요 잘 누르는 사람은 아닌데\n이...,0
98,2:45 아련...,5


### 자동화
- 시간을 트리거로 작동
- 하나의 셀에 모든 코드들을 다 합치기 -> 함수화 -> 스케줄 사용

In [2]:
def youtube_comments():
    """리뷰와 좋아요 수를 수집 후, DataFrame 형변환"""
    # 유튜브 - 사나고 영상 페이지
    url = "https://www.youtube.com/watch?v=vEIjQYiiOvM&t=8s"
    
    # 테스트 브라우저 열기
    driver = wb.Chrome()
    driver.get(url)
    
    driver.maximize_window()
    
    # 모든 댓글을 수집하기에는 시간이 많이 걸림...
    # 5번만 스크롤 하강 수행
    
    for i in range(5) : 
        body = driver.find_element(By.TAG_NAME, "body")
        body.send_keys(Keys.END)
        time.sleep(2)
    
    # 지정할 때 copy selector를 쓰던, 수기로 작성하던 항상 결과 확인!
    driver.find_elements(By.CSS_SELECTOR, "yt-attributed-string#content-text")[0].text
    
    #content-text
    
    # 리뷰와 좋아요 요소 지정
    reviews=driver.find_elements(By.CSS_SELECTOR, "yt-attributed-string#content-text")
    likes = driver.find_elements(By.CSS_SELECTOR, "span#vote-count-middle")
    
    # 1. 좋아요가 없으면 0으로 채우기
    # 2. 단위로 글자가 들어가 있는 경우 변환
    
    reviews_list=[]
    like_list=[]
    
    #추출
    for i in range(len(reviews)):
        reviews_list.append(reviews[i].text) #해당 리뷰 쌓아주기
        # 좋아요가 없으면 0으로 채우기
        if likes[i].text == "": #해당 리뷰에 좋아요가 없을때
            like_list.append(0) #0으로 대체해서 쌓아주기
        else:
            try: #좋아요를 형변환(정수)해서 쌓아주기
                like_list.append(int(likes[i].text)) #정수로 형변환 시도해보기!
            except: #좋아요가 숫자값이 아닌 경우!
                like_list.append(int(float(likes[i].text.replace("천",""))*1000))
    
    re_dict={
        "리뷰":reviews_list,
        "좋아요 수":like_list
    }
    df= pd.DataFrame(re_dict)
    driver.quit()
    return df

In [3]:
# 파일로 저장하는 함수
def save_to_csv(dataframe, filename): 
    """csv저장""" #독스트링, 주석임
    dataframe.to_csv(filename,index=False, encoding="utf-8")
    print(f"{filename} 저장 완료")

In [4]:
# 작업 정의
def job():
    print("작업을 시작합니다.")
    data= youtube_comments()
    save_to_csv(data,filename=f"youtube_comments_{time.strftime("%Y%m%d_%H%M%S")}.csv") #매번 다른 파일명으로 저장
    print("작업을 완료하였습니다.")

In [36]:
time.strftime("%Y%m%d_%H%M%S")

'20241108_102008'

#### Schedule Library

In [39]:
# !pip install schedule

In [5]:
import schedule
#1. 스케줄에 작업을 등록
schedule.every().day.at("10:35").do(job)

# every(): 일정간격마다 작업을 수행하겠다는 스케줄 설정
# every(5): 5초마다 실행

# day: 하루하루 반복
# at: 시간대 설정
#do(): 실행시킬 함수, 모듈

Every 1 day at 10:35:00 do job() (last run: [never], next run: 2024-11-08 10:35:00)

In [ ]:
# 2. 스케줄대로 작업을 수행
while True:
    schedule.run_pending() # 현 시점에서 실행 될 준비가 된 작업을 실행
    time.sleep(60)

작업을 시작합니다.
youtube_comments_20241108_103613.csv 저장 완료
작업을 완료하였습니다.
